## Exercise Set 1

### Multiple Choice
**Q1.** Which of these are NOT key components for an optimization model?
- decision variables
- parameters
- objective function
- hyperparameters

**Q2.** MO can be used as a strict replacement for nearly all ML problems
- True
- False

**Q3.** ML can be used as a strict replacement for nearly all MO problems
- True
- False

**Q4.** Which of the following is a type of decision variables in mathematical optimization?
- binary
- complex
- feature
- complimentary

**Q5.** To solve a mixed-integer program, a solver may need to solve this many linear programs:
- none
- the number of decision variables
- the number of total constraints
- possibly exponentially many

**Q6.** It is required that you or your data analytics team develop an expertise in `predictive` analytics before attempting `prescriptive` analytics (like mathematical optimization).
- True
- False

**Q7.** The `feasible region` of a linear program (LP) will have ____ points in it than its corresponding mixed-integer program (MIP), assuming the two models are exactly the same other than the LP has only continuous variables and the MIP contains integer variables.
- more
- less
- always exactly the same

**Q8.** Unless specified, the default variable type when using `addVars()` is **continuous**.
- True
- False

Let $J = \{\texttt{Apple, Banana, Coconut, Dragonfruit, Elderberry, Fig, Gooseberry}\}$ and $T = \{1, 2, 3, 4\}$

**Q9-a.** Adding decision variables using `addVars(J,...)` and `addVars(range(8),...)` will add the *same* number of variables to a model
- True
- False

**Q9-b.** Using the sets above, adding decision variables using `addVars(J, T,...)` and `addVars(range(28),...)` will add the *same* number of variables to a model
- True
- False

### Formulation and Coding
Below is code for the entire original model in one cell if you would like to use it to help with these exercises.

In [2]:
%pip install gurobipy

import pandas as pd
import gurobipy as gp
from gurobipy import GRB


#----------------------------#
# Sets
#----------------------------#
production = ['Baltimore','Cleveland','Little Rock','Birmingham','Charleston']
distribution = ['Columbia','Indianapolis','Lexington','Nashville','Richmond','St. Louis'] # Demand

path = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/Modeling_Session_1/'

#----------------------------#
# Parameters
#----------------------------#

# Transportation Cost
transp_cost = pd.read_csv(path + 'cost.csv', index_col=[0,1]).squeeze("columns")

# Max production on facility p
max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")

# Demand on facility d
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand")

# Production on p must not exceed frac capacity
frac = 0.75

#----------------------------#
# Model creation and decision variables
#----------------------------#

# We first create the model object
m = gp.Model('widgets')

# Then we add decision variables
x = m.addVars(production, distribution, name = 'prod_ship')

#----------------------------#
# Then we can add constraints
#----------------------------#

# The sum of widgets produce on p, and ship to d must meet demand in d forall d
meet_demand = m.addConstrs((gp.quicksum(x[p,d] for p in production) >= n_demand[d] for d in distribution), name = 'meet_demand')


# The sum of the widgets produce on p, and ship to d must be below the max production of p for all p
can_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production), name = 'can_produce')

# The sum of the widgets produce on p, and ship to d must be at least frac of the capacity of production on p
must_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production), name = 'must_produce')


# minimize sum of p sum of d trasnp_cost[i,j]*x[i,j]
m.setObjective(gp.quicksum(transp_cost[i,j]*x[i,j] for i in production for j in distribution), GRB.MINIMIZE)
m.optimize()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 72.7 MB/s eta 0:00:00
Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 16 rows, 30 columns and 90 nonzeros
Model fingerprint: 0x20186c14
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 2e+02]
Presolve removed 5 rows and 0 columns
Presolve time: 0.01s
Presolved: 11 rows, 35 columns, 65 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.610000e+02   0.000000e+00      0s
      15    1.7048900e+03   0.000000e+00   0.000000e+00      0s

Solved in 15 iterations and 0.02 seconds (0.00 work units)
Optimal obje

In [3]:
m.write('widgets.rlp')
#m.display()

In [4]:
x_values = pd.Series(m.getAttr('x', x), name = 'shipment', index=transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)
sol

cost  shipment
production  distribution                
Baltimore   Columbia      4.50       0.0
            Indianapolis  5.09       0.0
            Lexington     4.33       0.0
            Nashville     5.96      19.0
            Richmond      1.96     116.0
            St. Louis     7.30       0.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
            Lexington     2.54       0.0
            Nashville     4.13       2.0
            Richmond      3.20       0.0
            St. Louis     4.88       0.0
Little Rock Columbia      6.42       0.0
            Indianapolis  4.83       0.0
            Lexington     3.39       0.0
            Nashville     4.40       0.0
            Richmond      7.44       0.0
            St. Louis     2.92     140.0
Birmingham  Columbia      3.33       0.0
            Indianapolis  4.33       0.0
            Lexington     3.38       0.0
            Nashville     1.53      80.0
            Richmond      5.95       0.0
            St. Louis     4.01       0.0
Charleston  Columbia      3.02       0.0
            Indianapolis  2.61       0.0
            Lexington     1.61     121.0
            Nashville     4.44       0.0
            Richmond      2.36       0.0
            St. Louis     4.60      41.0

In [5]:
sol[sol.shipment > 0]

cost  shipment
production  distribution                
Baltimore   Nashville     5.96      19.0
            Richmond      1.96     116.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
            Nashville     4.13       2.0
Little Rock St. Louis     2.92     140.0
Birmingham  Nashville     1.53      80.0
Charleston  Lexington     1.61     121.0
            St. Louis     4.60      41.0

In [7]:
all_vars = {v.varName: v.x for v in m.getVars()}
all_vars

{'prod_ship[Baltimore,Columbia]': 0.0,
 'prod_ship[Baltimore,Indianapolis]': 0.0,
 'prod_ship[Baltimore,Lexington]': 0.0,
 'prod_ship[Baltimore,Nashville]': 19.0,
 'prod_ship[Baltimore,Richmond]': 116.0,
 'prod_ship[Baltimore,St. Louis]': 0.0,
 'prod_ship[Cleveland,Columbia]': 89.0,
 'prod_ship[Cleveland,Indianapolis]': 95.0,
 'prod_ship[Cleveland,Lexington]': 0.0,
 'prod_ship[Cleveland,Nashville]': 2.0,
 'prod_ship[Cleveland,Richmond]': 0.0,
 'prod_ship[Cleveland,St. Louis]': 0.0,
 'prod_ship[Little Rock,Columbia]': 0.0,
 'prod_ship[Little Rock,Indianapolis]': 0.0,
 'prod_ship[Little Rock,Lexington]': 0.0,
 'prod_ship[Little Rock,Nashville]': 0.0,
 'prod_ship[Little Rock,Richmond]': 0.0,
 'prod_ship[Little Rock,St. Louis]': 140.0,
 'prod_ship[Birmingham,Columbia]': 0.0,
 'prod_ship[Birmingham,Indianapolis]': 0.0,
 'prod_ship[Birmingham,Lexington]': 0.0,
 'prod_ship[Birmingham,Nashville]': 80.0,
 'prod_ship[Birmingham,Richmond]': 0.0,
 'prod_ship[Birmingham,St. Louis]': 0.0,
 'prod_shi

In [9]:
xvals = {k: v.x for k,v in x.items() if v.x>0}
xvals

{('Baltimore', 'Nashville'): 19.0,
 ('Baltimore', 'Richmond'): 116.0,
 ('Cleveland', 'Columbia'): 89.0,
 ('Cleveland', 'Indianapolis'): 95.0,
 ('Cleveland', 'Nashville'): 2.0,
 ('Little Rock', 'St. Louis'): 140.0,
 ('Birmingham', 'Nashville'): 80.0,
 ('Charleston', 'Lexington'): 121.0,
 ('Charleston', 'St. Louis'): 41.0}

**Solution Analysis**

In [11]:
ship_out = sol.groupby('production')['shipment'].sum()
ship_out

,shipment
production,
Baltimore,135.0
Birmingham,80.0
Charleston,162.0
Cleveland,186.0
Little Rock,140.0


In [13]:
pd.DataFrame({'Remaining' :max_prod-ship_out, 'Utilization': ship_out/max_prod})

,Remaining,Utilization
Baltimore,45.0,0.75
Birmingham,0.0,1.00
Charleston,18.0,0.90
Cleveland,14.0,0.93
Little Rock,0.0,1.00


You are told there is a new policy for transporting **widgets** from production facilities. It is now required that the minimum number of widgets shipped from any production facility to any distribution center needs to be at least 20.

**Q10-a.** Write out how the formulation changes using mathematical notation given the new requirement.

**Q10-b.** Write the changes for **Q10-a** in gurobipy code in the cell below (no need to run it, unless you want to copy the model here to check).

In [16]:
#----------------------------#
#    ADD constraint of at least 20 widgets
#----------------------------#
# Binary "lane used" variables
y = m.addVars(production, distribution, vtype=GRB.BINARY, name="lane_used")

MIN_LOT = 20

# Big-M upper bounds per lane (tight as possible)
U = {(p,d): min(max_prod[p], n_demand[d]) for p in production for d in distribution}

# Min-lot policy: if the lane is used, ship at least 20; otherwise ship 0
m.addConstrs((x[p,d] >= MIN_LOT * y[p,d] for p in production for d in distribution),
             name="min_lot_if_used")
m.addConstrs((x[p,d] <= U[p,d]     * y[p,d] for p in production for d in distribution),
             name="shut_lane_when_off")

#--------------------
#.  SOLVE
# -------------
m.optimize()

# results
x_values_atleast_20 = pd.Series(m.getAttr('x', x), name = 'shipment', index=transp_cost.index)
sol_atleast_20 = pd.concat([transp_cost, x_values_atleast_20], axis=1)
sol_atleast_20 = sol[sol.shipment > 0]
sol_atleast_20

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 196 rows, 120 columns and 450 nonzeros
Variable types: 30 continuous, 90 integer (90 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e+00, 7e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+01, 2e+02]

MIP start from previous solve produced solution with objective 1708.55 (0.01s)
Loaded MIP start from previous solve with objective 1708.55

Presolve removed 120 rows and 60 columns
Presolve time: 0.00s
Presolved: 76 rows, 60 columns, 210 nonzeros
Variable types: 30 continuous, 30 integer (30 binary)

Root relaxation: objective 1.704890e+03, 24 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth

cost  shipment
production  distribution                
Baltimore   Nashville     5.96      19.0
            Richmond      1.96     116.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
            Nashville     4.13       2.0
Little Rock St. Louis     2.92     140.0
Birmingham  Nashville     1.53      80.0
Charleston  Lexington     1.61     121.0
            St. Louis     4.60      41.0

In [17]:
ship_out_20 = sol_atleast_20.groupby('production')['shipment'].sum()
ship_out_20

,shipment
production,
Baltimore,135.0
Birmingham,80.0
Charleston,162.0
Cleveland,186.0
Little Rock,140.0


In [18]:
pd.DataFrame({'Remaining' :max_prod-ship_out, 'Utilization': ship_out_20/max_prod})

,Remaining,Utilization
Baltimore,45.0,0.75
Birmingham,0.0,1.00
Charleston,18.0,0.90
Cleveland,14.0,0.93
Little Rock,0.0,1.00


The initial widget model $m$ represented a single time period (e.g. a week, month, quarter). Suppose we added a time component using a set $T = \{0, 1, 2, 3\}$ representing a quarter of a year.

**Q11-a.** Use `addVar()` or `addVars()` to create a decision variable in gurobipy that represents the number of widgets shipped from a production facility to a distribution center for a given time period.

**Q11-b.** To reference a time period before a given time $t$, we can use $t-1$ as a subscript (since $T$ is a set of integers), but $t-1$ doesn't work when $t = 0$ since $-1$ isn't in $T$. Fill in the ??? in the code below to represent a set of constraints that limits the amount a production facility can produce to `max_prod[p]` over *two consecutive* time periods.

In [ ]:
time_prod_limit = m.addConstrs((gp.quicksum(x[p,d,???] + x[p,d,???] for d in distribution) <= max_prod[p] for p in production for t in ??? if t ???), name = 'time_prod_limit')